In [1]:
import pandas as pd
import numpy as np
import sklearn
import os
import geopandas as gpd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from catboost import CatBoostRegressor
from sklearn.preprocessing import FunctionTransformer

from jupyter_dash import JupyterDash
import dash
from dash import html, Input, Output, dcc
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objs import Scatter, Figure, Layout
import json

# Подгружаем подготовленные признаки и обученную модель

In [2]:
to_predict = pd.read_csv(os.path.join('data', 'set_for_dash.csv'))
gdf = gpd.read_file('mo.geojson')

In [3]:
cat_model = CatBoostRegressor()

cat_model.load_model('catboost_model.cbm')

In [4]:
area_rus = {'Ajeroport': 'Аэропорт',
 'Akademicheskoe': 'Академический',
 'Alekseevskoe': 'Алексеевский',
 "Altuf'evskoe": 'Алтуфьевский',
 'Arbat': 'Арбат',
 'Babushkinskoe': 'Бабушкинский',
 'Basmannoe': 'Басманный',
 'Begovoe': 'Беговой',
 'Beskudnikovskoe': 'Бескудниковский',
 'Bibirevo': 'Бибирево',
 'Birjulevo Vostochnoe': 'Бирюлёво Восточное',
 'Birjulevo Zapadnoe': 'Бирюлёво Западное',
 'Bogorodskoe': 'Богородское',
 'Brateevo': 'Братеево',
 'Butyrskoe': 'Бутырский',
 'Caricyno': 'Царицыно',
 'Cheremushki': 'Черёмушки',
 "Chertanovo Central'noe": 'Чертаново Центральное',
 'Chertanovo Juzhnoe': 'Чертаново Южное',
 'Chertanovo Severnoe': 'Чертаново Северное',
 'Danilovskoe': 'Даниловский',
 'Dmitrovskoe': 'Дмитровский',
 'Donskoe': 'Донской',
 'Dorogomilovo': 'Дорогомилово',
 'Filevskij Park': 'Филёвский Парк',
 'Fili Davydkovo': 'Фили-Давыдково',
 'Gagarinskoe': 'Гагаринский',
 "Gol'janovo": 'Гольяново',
 'Golovinskoe': 'Головинский',
 'Hamovniki': 'Хамовники',
 'Horoshevo-Mnevniki': 'Хорошёво-Мнёвники',
 'Horoshevskoe':'Хорошёвский',
 'Hovrino': 'Ховрино',
 'Ivanovskoe': 'Ивановское',
 'Izmajlovo': 'Измайлово',
 'Jakimanka': 'Якиманка',
 'Jaroslavskoe': 'Ярославский',
 'Jasenevo': 'Ясенево',
 'Juzhnoe Butovo': 'Южное Бутово',
 'Juzhnoe Medvedkovo': 'Южное Медведково',
 'Juzhnoe Tushino': 'Южное Тушино',
 'Juzhnoportovoe': 'Южнопортовый',
 'Kapotnja': 'Капотня',
 "Kon'kovo": 'Коньково',
 'Koptevo': 'Коптево',
 'Kosino-Uhtomskoe': 'Косино-Ухтомский',
 'Kotlovka': 'Котловка',
 "Krasnosel'skoe": 'Красносельский',
 'Krjukovo': 'Крюково',
 'Krylatskoe': 'Крылатское',
 'Kuncevo': 'Кунцево',
 'Kurkino': 'Куркино',
 "Kuz'minki": 'Кузьминки',
 'Lefortovo': 'Лефортово',
 'Levoberezhnoe': 'Левобережный',
 'Lianozovo': 'Лианозово',
 'Ljublino': 'Люблино',
 'Lomonosovskoe': 'Ломоносовский',
 'Losinoostrovskoe': 'Лосиноостровский',
 "Mar'ina Roshha": 'Марьина Роща',
 "Mar'ino": 'Марьино',
 'Marfino': 'Марфино',
 'Matushkino': 'Матушкино',
 'Meshhanskoe': 'Мещанский',
 'Metrogorodok': 'Метрогородок',
 'Mitino': 'Митино',
 'Molzhaninovskoe': 'Молжаниновский',
 "Moskvorech'e-Saburovo": 'Москворечье-Сабурово',
 'Mozhajskoe': 'Можайский',
 'Nagatino-Sadovniki': 'Нагатино-Садовники',
 'Nagatinskij Zaton': 'Нагатинский Затон',
 'Nagornoe': 'Нагорный',
 'Nekrasovka': 'Некрасовка',
 'Nizhegorodskoe': 'Нижегородский',
 'Novo-Peredelkino': 'Ново-Переделкино',
 'Novogireevo': 'Новогиреево',
 'Novokosino': 'Новокосино',
 'Obruchevskoe': 'Обручевский',
 'Ochakovo-Matveevskoe': 'Очаково-Матвеевское',
 'Orehovo-Borisovo Juzhnoe': 'Орехово-Борисово Южное',
 'Orehovo-Borisovo Severnoe': 'Орехово-Борисово Северное',
 'Ostankinskoe': 'Останкинский',
 'Otradnoe': 'Отрадное',
 'Pechatniki': 'Печатники',
 'Perovo': 'Перово',
 'Pokrovskoe Streshnevo': 'Покровское-Стрешнево',
 'Poselenie Desjonovskoe': 'Десёновское',
 'Poselenie Filimonkovskoe': 'Филимонковское',
 'Poselenie Kievskij': 'Киевский',
 'Poselenie Klenovskoe': 'Клёновское',
 'Poselenie Kokoshkino': 'Кокошкино',
 'Poselenie Krasnopahorskoe': 'Краснопахорское',
 'Poselenie Marushkinskoe': 'Марушкинское',
 'Poselenie Mihajlovo-Jarcevskoe': 'Михайлово-Ярцевское',
 'Poselenie Moskovskij': 'Московский',
 'Poselenie Mosrentgen': '"Мосрентген"',
 'Poselenie Novofedorovskoe': 'Новофёдоровское',
 'Poselenie Pervomajskoe': 'Первомайское',
 'Poselenie Rjazanovskoe': 'Рязановское',
 'Poselenie Rogovskoe': 'Роговское',
 'Poselenie Shhapovskoe': 'Щаповское',
 'Poselenie Shherbinka': 'Щербинка',
 'Poselenie Sosenskoe': 'Сосенское',
 'Poselenie Vnukovskoe': 'Внуковское',
 'Poselenie Voronovskoe': 'Вороновское',
 'Poselenie Voskresenskoe': 'Воскресенское',
 'Preobrazhenskoe': 'Преображенское',
 'Presnenskoe': 'Пресненский',
 'Prospekt Vernadskogo': 'Проспект Вернадского',
 'Ramenki': 'Раменки',
 'Rjazanskij': 'Рязанский',
 'Rostokino': 'Ростокино',
 'Savelki': 'Савёлки',
 'Savelovskoe': 'Савёловский',
 'Severnoe': 'Северный',
 'Severnoe Butovo': 'Северное Бутово',
 'Severnoe Izmajlovo': 'Северное Измайлово',
 'Severnoe Medvedkovo': 'Северное Медведково',
 'Severnoe Tushino': 'Северное Тушино',
 'Shhukino': 'Щукино',
 'Silino': 'Силино',
 'Sokol': 'Сокол',
 "Sokol'niki": 'Сокольники',
 'Sokolinaja Gora': 'Соколиная Гора',
 'Solncevo': 'Солнцево',
 'Staroe Krjukovo': 'Старое Крюково',
 'Strogino': 'Строгино',
 'Sviblovo': 'Свиблово',
 'Taganskoe': 'Таганский',
 "Tekstil'shhiki": 'Текстильщики',
 'Teplyj Stan': 'Тёплый Стан',
 'Timirjazevskoe': 'Тимирязевский',
 'Troickij okrug': 'Троицк',
 'Troparevo-Nikulino': 'Тропарёво-Никулино',
 'Tverskoe': 'Тверской',
 'Veshnjaki': 'Вешняки',
 'Vnukovo': 'Внуково',
 'Vojkovskoe': 'Войковский',
 'Vostochnoe': 'Восточный',
 'Vostochnoe Degunino': 'Восточное Дегунино',
 'Vostochnoe Izmajlovo': 'Восточное Измайлово',
 'Vyhino-Zhulebino': 'Выхино-Жулебино',
 "Zamoskvorech'e": 'Замоскворечье',
 'Zapadnoe Degunino': 'Западное Дегунино',
 'Zjablikovo': 'Зябликово',
 'Zjuzino': 'Зюзино'}

In [5]:
fig = px.choropleth_mapbox(gdf,
                           geojson=gdf.geometry,
                           locations=gdf.index,
                           color="NAME",
                           center={"lat": 55.7385, "lon": 37.6093},
                           mapbox_style="open-street-map",
                           zoom=8.5)

# Запускаем Dash приложение

In [6]:
app = JupyterDash(__name__)
app.layout = html.Div([

    html.H2("Введите параметры для предсказания стоимости", style={'text-align': 'center'}),

    html.Div([
                html.Br(),
                html.Br(),

                html.Div([
                    "Внесите информацию об общей площади: ",
                    dcc.Input(id='full_sq', value='общая', type='text')
                ]),

                html.Br(),
                html.Br(),
                html.Br(),

                html.Div([
                    "Внесите информацию о годе постройки: ",
                    dcc.Input(id='build_year', value='год', type='text')
                ]),

                html.Br(),
                html.Br(),
                html.Br(),

                html.Div([
                        "Выберите тип объекта:",
                        dcc.Dropdown(
                            to_predict.product_type.unique(),
                            id='product_type',
                            style={'backgroundColor': 'white'}
                        )
                        ]),

                html.Br(),
                html.Br(),
                html.Br(),

                html.Div(id='my-output',
                         style={'float': 'left',
                                'font-weight': 'bold',
                                'display': 'inline-block'})

              ], style={'width': '28%', 'float': 'left', 'display': 'flexbox'}),

              html.Div([

                        html.Div([
                                "Выберите район:",
                                dcc.Graph(figure=fig,
                                          id='sub_area')
                            ], style={'width': '55%', 'marginLeft': '40%'})
                      ], style={'width': '100%', 'float': 'topright'}
                     )

])

@app.callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='full_sq', component_property='value'),
    Input(component_id='build_year', component_property='value'),
    Input('product_type', 'value'),
    Input('sub_area', 'clickData')
)
def update_output_div(input_value1, input_value2, product_type, clickData):
    sub_area = None
    if clickData:
        for i, j in area_rus.items():
            if j == gdf.NAME[clickData['points'][0]['curveNumber']]:
                sub_area = i
    if input_value1.isdigit() and input_value2.isdigit() and product_type and sub_area:
        if product_type == 'Investment':
            if to_predict[(to_predict.product_type == product_type) &
                                    (to_predict.sub_area == sub_area)].empty:
                df = to_predict[(to_predict.product_type == 'OwnerOccupier') &
                                    (to_predict.sub_area == sub_area)]
                df.product_type = product_type
            else:
                df = to_predict[(to_predict.product_type == product_type) &
                                    (to_predict.sub_area == sub_area)]
        else:
            if to_predict[(to_predict.product_type == product_type) &
                                    (to_predict.sub_area == sub_area)].empty:
                df = to_predict[(to_predict.product_type == 'Investment') &
                                    (to_predict.sub_area == sub_area)]
                df.product_type = product_type
            else:
                df = to_predict[(to_predict.product_type == product_type) &
                                    (to_predict.sub_area == sub_area)]

        df.full_sq = int(input_value1)
        df.build_year = int(input_value2)


        if (int(input_value1) <= 12 or int(input_value2) < 1900 or int(input_value2) > 2015) and product_type and sub_area:
            return f'Вы выбрали площадь {input_value1} кв. м и год постройки {input_value2}. Попробуйте указать площадь больше 12 кв.м и год постройки в интервале от 1900 до 2015'
        else:
            return f'Предполагаемая стоимость квартиры в районе {area_rus[sub_area]}: {int(np.exp(cat_model.predict(df)))}'


app.run_server(debug=True, use_reloader=False)

Dash app running on http://127.0.0.1:8050/
